# Talking vs. Others

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import specgram
from scipy.signal import spectrogram
from sklearn.model_selection import train_test_split

In [2]:
eeg_df = pd.read_csv("../data/2018-10-12-preston_walk_raw.csv", skiprows=15)

In [3]:
eeg_df.head()

,Time,LE,F4,C4,PO8,PO7,C3,F3,Trigger,Time_Offset,ADC_Status,ADC_Sequence,Event,Comments
0,0.0033,3642.0,-2536.8,3530.1,2459.4,296.7,3454.5,-1103.4,0,6,0,207,0,Start Data Acquisition
1,0.0067,3635.7,-2526.3,3531.0,2451.9,299.7,3459.6,-1098.9,0,12,0,208,0,NaN
2,0.0100,3632.7,-2535.9,3528.6,2441.4,302.1,3454.8,-1103.4,0,18,0,209,0,NaN
3,0.0133,3626.7,-2535.3,3524.1,2450.1,297.9,3455.1,-1097.1,0,24,0,210,0,NaN
4,0.0167,3626.4,-2527.8,3539.4,2469.0,289.8,3452.7,-1089.9,0,30,0,211,0,NaN


## Filter data

In [4]:
from mne.filter import filter_data

In [5]:
channels = eeg_df.columns[1:8] 

In [6]:
eeg_data = eeg_df[channels].values

In [7]:
filtered = filter_data(eeg_data.T, sfreq=300, l_freq=1, h_freq=40)

Setting up band-pass filter from 1 - 40 Hz
l_trans_bandwidth chosen to be 1.0 Hz
h_trans_bandwidth chosen to be 10.0 Hz
Filter length of 991 samples (3.303 sec) selected


In [8]:
eeg_df[channels] = filtered.T

## Epoch data

In [9]:
events = eeg_df.Event.nonzero()[0]

In [10]:
events

array([  1446,   2970,   7952,  47145,  78073, 135086])

In [11]:
walk_one_start = events[2] + 300
walk_one_stop = events[3] - 300
typing_start = events[3] + 300
typing_stop = events[4] - 300
talking_start = events[4] + 300  
talking_stop = events[5] - 300  
walk_two_start = events[5] + 300
walk_two_stop = len(eeg_df) - 9000

In [12]:
typing_data = eeg_df.loc[typing_start : typing_stop][channels]
talking_data = eeg_df.loc[talking_start : talking_stop][channels]
walk_one_data = eeg_df.loc[walk_one_start : walk_one_stop][channels]
walk_two_data = eeg_df.loc[walk_two_start : walk_two_stop][channels]

In [13]:
typing_data = typing_data[: len(typing_data) // 300 * 300]
talking_data = talking_data[: len(talking_data) // 300 * 300]
walk_one_data = walk_one_data[: len(walk_one_data) // 300 * 300]
walk_two_data = walk_two_data[: len(walk_two_data) // 300 * 300]

## Power, create data

In [14]:
other_data = np.concatenate((typing_data, walk_one_data, walk_two_data), axis=0) 

In [15]:
f, t, spec = spectrogram(talking_data.T, fs=300, nperseg=600, noverlap=300)

In [16]:
talking_spec = spec[:, :81, :].reshape(-1, spec.shape[2]).T

In [17]:
f, t, spec = spectrogram(other_data.T, fs=300, nperseg=600, noverlap=300)

In [18]:
other_spec = spec[:, :81, :].reshape(-1, spec.shape[2]).T

In [19]:
X = np.concatenate((talking_spec, other_spec), axis=0)

In [20]:
y = np.concatenate((np.ones(len(talking_spec)), np.zeros(len(other_spec))), axis=0)

In [21]:
# shuffle data to get train/test
args = np.arange(len(X))
np.random.shuffle(args)
X = X[args]
y = y[args]

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

## SVM, Hyperparameter tuning (Walking vs. others)

In [69]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

In [70]:
Cs = [0.001, 0.01, 0.1, 1, 5, 10]
degrees = [1, 2, 3, 4, 5, 10, 15, 20]
gammas = np.linspace(0.01, 50.01, 10)

In [71]:
hyper_params = {"C":Cs, "gamma":gammas}

In [72]:
grid = GridSearchCV(SVC(kernel='rbf'), param_grid=hyper_params, cv=KFold(n_splits=5), verbose=True)

In [73]:
grid.fit(X_train, y_train)

Fitting 5 folds for each of 60 candidates, totalling 300 fits


[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:   59.8s finished


GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
       error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [0.001, 0.01, 0.1, 1, 5, 10], 'gamma': array([1.00000e-02, 5.56556e+00, 1.11211e+01, 1.66767e+01, 2.22322e+01,
       2.77878e+01, 3.33433e+01, 3.88989e+01, 4.44544e+01, 5.00100e+01])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=True)

In [74]:
grid.best_estimator_

SVC(C=0.001, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.01, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [90]:
svc = SVC(kernel='poly', C=0.1, degree=1)
svc.fit(X_train, y_train)

SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=1, gamma='auto', kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [91]:
svc.score(X_train, y_train)

1.0

In [92]:
svc.score(X_test, y_test)

0.9215686274509803

### confusion matrix: 
| type | type |
|--------------|--------------|
|True Positive | False Positive|
|False Negative| True Negative|

In [44]:
from sklearn.metrics import confusion_matrix

In [93]:
confusion_matrix(y_test, svc.predict(X_test))

array([[26,  2],
       [ 2, 21]])